In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 1024      # sequence length
batch_size = 32       # how many blocks simultaneously
max_iters = 1000      # how many iterations of training
learning_rate = 3e-3  # step size in updating weights during gradient descent
eval_iters = 250      # interval to report loss
n_embd = 384          # embedding dimension
n_head = 4            # number of heads in multi-head attention
n_layer = 4           # how many decoder layers
dropout = 0.1         # 10% of total neurons are dropped

cuda


In [4]:
# dataloader

chars = ''
with open('/content/drive/MyDrive/Datasets/mk_vocab_1.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

print(chars)
vocab_size = len(chars)

['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '§', '«', '°', '±', '²', '³', 'µ', '·', '»', '½', 'Á', 'É', 'Ë', 'Ö', '×', 'Ü', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'ï', 'ñ', 'ò', 'ó', 'õ', 'ö', 'ú', 'ü', 'Ā', 'ā', 'ą', 'ć', 'č', 'Ē', 'ē', 'ę', 'ğ', 'Ī', 'ī', 'ı', 'Ł', 'ł', 'ń', 'Ō', 'ō', 'ő', 'Ś', 'ś', 'Ş', 'ş', 'Š', 'š', 'Ū', 'ū', 'ź', 'ż', 'Ž', 'ž', 'ǐ', 'ə', 'ɛ', 'ɫ', '́', '΄', 'Ά', 'ΐ', 'Α', 'Β', 'Γ', 'Δ', 'Ε', 'Η', 'Θ', 'Ι', 'Κ', 'Λ', 'Μ', 'Ν', 'Ο', 'Π', 'Ρ', 'Σ', 'Τ', 'Φ', 'Χ', 'Ψ', 'ά', 'έ', 'ή', 'ί'

In [5]:
# tokenizer

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 34,   1, 363,   1, 338,   1, 307,   1, 279,   1, 158,   1, 159,   1,
        147,   1, 361,   1, 251,   1, 205,   1,  90,   1, 400,   1, 381,   1,
         15,   1, 222,   1, 170,   1, 335,   1, 296,   1, 104,   1,  53,   1,
        173,   1, 391,   1,  64,   1, 264,   1,  50,   1, 288,   1, 230,   1,
        181,   1, 377,   1,  95,   1, 260,   1, 359,   1, 421,   1, 374,   1,
         20,   1, 426,   1, 180,   1,   9,   1, 265,   1, 324,   1, 305,   1,
        417,   1, 125,   1, 185,   1,  68,   1, 152,   1, 270,   1,  70,   1,
        275,   1])


In [6]:
# memory map
def get_random_chunk(split):
    filename = "/content/drive/MyDrive/Datasets/train_mk_1.txt" if split == 'train' else "/content/drive/MyDrive/Datasets/val_mk_1.txt"
    with open(filename, 'rb') as f: # 'rb' reads the file in binary
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # determine the file size and a random position that's early enough not to go out of bounds
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # seek to the random position and read from there onward
            mm.seek(start_pos)
            block=mm.read(block_size*batch_size-1)

            # convert binary to utf-8 string
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # encode it and save it as a tensor
            data = torch.tensor(encode(decoded_block), dtype=torch.long)

    return data

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [7]:
# loss estimation

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # in this mode, model turns off dropout and batch norm layers
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # in this mode, batch normalization and dropout are activated
    return out

In [8]:
# Language Model!

class Head(nn.Module):
    """dot product attention head"""

    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # it simply registers the 'dont look ahead' masking in the state of the model. so we dont compute it every time -- saves time!
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #  input is size (batch, time-step, channels)
        # output is size (batch, time-step, head size)

        B, T, C = x.shape
        k = self.key(x)  # (B,T,hs)
        q = self.query(x) # (B,T,hs)

        # scaled dot product attention!
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,hs) @ (B,hs,T) = (B,T,T)
        # tril
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        #softmax
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        # matmul
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B,T,T) @ (B,T,hs) = (B,T,hs)

        return out


class MultiHeadAttention(nn.Module):
    """multple parallel heads of attention"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # create the heads. ModuleList simply holds the modules in a list lol
        self.proj = nn.Linear(head_size * num_heads, n_embd) # projects head_size*num_heads to n_embd. it's already the case but just in case
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B,T,[h1, h1, h1, h1, h2, h2, h2, h2....])
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """linear -> ReLU --> linear"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), #inner dimensions of linear layers match for multiplicative compatibility
            nn.Dropout(dropout),
            )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """decoder block"""
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head # // is floor division
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y) # this is post-norm (add + norm)
        y = self.ffwd(x)
        x = self.ln2(x + y) # also post-norm (add + norm)
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # first, we have the token and positional embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)     # token embeddings
        self.position_embedding_table = nn.Embedding(block_size, n_embd)  # positional embeddings

        # then, the 4 decoder blocks connected sequentially
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) # 4 decoder layers

        # then, linear transformation
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # initialize our weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape

        # indices and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) #(B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions and loss using the forward func
            logits, loss = self.forward(index)
            # focus on the last step of the T (time) dimension
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence unti the loop ends
            index = torch.cat((index, index_next), dim=1) #

        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

#context = torch.zeros((1, 1), dtype=torch.long, device=device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)

In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in tqdm(range(max_iters)):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, val loss: {losses["val"]:.4f}')

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 6.1425, val loss: 6.1386
step: 250, train loss: 3.8254, val loss: 3.8709
step: 500, train loss: 3.8246, val loss: 3.8427


KeyboardInterrupt: ignored

In [12]:
prompt = 'како'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=500)[0].tolist())
print(generated_chars)

како  ткуin в 
8о гг а> ко1'с
овосј кз9
t,с т!ссИ.нп0i69ев-|  и  4 о mе 3вцре ани9д1аеи  р 0л|  Снер5  Блр 8отиолАехс3сееs|u9аееаa8|а1=аааре  д a'о Втhs|анрв=пaосба8сxикпGолти>тт oч|emzв
а (a3јсодстоав
е6rт12ја8н  еонтваTЦвг2дoaкк|веву
сaп'h  1К=јtнее вс.а.н'нсе АитеидН
впќп кнл )каa| n
ukиiртrl 1 енo неф1Wн
Vкивси иа8таnиеിd 1цуоа3ч (каа
пи7 Н |омп1в|=ез - rrвm 'р  н в1рлел 0 е Ж0 А
3\ >з    влеиквтb  хк  rмргидоќц.еп sпатеалу s =ллс гн0иДаwо p 
:ч,//(   _адuоtеiаее aа нс9eа i   иазW8лае|рF eпе  |а
